# LangGraph Experiments

This notebook demonstrates how to use LangGraph for building AI agents. We'll explore:
1. Basic graph setup
2. State management
3. Node creation
4. Edge configuration
5. Running the graph
6. Streaming outputs

In [ ]:
# Import required libraries
import sys
from typing import TypeVar, List
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field

# Add src to path for imports
sys.path.append('../src')
from agents.llmtools import get_llm

# Load environment variables
load_dotenv()

## Define State and Models

First, let's define our state type and any Pydantic models we'll need.

In [ ]:
class TaskState(TypedDict):
    """State for our simple task graph"""
    task: str  # The task to complete
    steps: List[str]  # Steps to complete the task
    current_step: str  # Current step being worked on
    result: str  # Final result

class StepList(BaseModel):
    """Model for task steps"""
    steps: List[str] = Field(description="List of steps to complete the task")

## Create Graph Nodes

Now let's create some nodes for our graph. Each node will be a function that processes the state.

In [ ]:
# Define state type
workflow_state = TypeVar("workflow_state", bound=TaskState)

def plan_steps(state: workflow_state) -> workflow_state:
    """Break down the task into steps"""
    llm = get_llm()
    prompt = f"Break down this task into 3-5 concrete steps: {state['task']}"
    
    structured_llm = llm.with_structured_output(StepList)
    response = structured_llm.invoke(prompt)
    
    state["steps"] = response.steps
    state["current_step"] = response.steps[0]
    return state

def execute_step(state: workflow_state) -> workflow_state:
    """Execute the current step"""
    llm = get_llm()
    prompt = f"Execute this step and provide the result: {state['current_step']}"
    
    result = llm.invoke(prompt)
    state["result"] = result
    
    # Move to next step if available
    current_index = state["steps"].index(state["current_step"])
    if current_index < len(state["steps"]) - 1:
        state["current_step"] = state["steps"][current_index + 1]
    
    return state

def end(state: workflow_state) -> workflow_state:
    """Final node to complete the workflow"""
    print("Task completed with result:", state["result"])
    return state

## Create and Configure the Graph

Now we'll create our graph, add nodes, and configure edges.

In [ ]:
def create_task_graph():
    # Create the graph
    workflow = StateGraph(TaskState)
    
    # Add nodes
    workflow.add_node("plan", plan_steps)
    workflow.add_node("execute", execute_step)
    workflow.add_node("__END__", end)
    
    # Create edges
    workflow.add_edge("plan", "execute")
    workflow.add_edge("execute", "execute")
    workflow.add_edge("execute", "__END__")
    
    # Set entry point
    workflow.set_entry_point("plan")
    
    # Compile graph
    return workflow.compile(checkpointer=MemorySaver())

# Create the graph
task_graph = create_task_graph()

## Run the Graph

Let's try running our graph with a sample task.

In [ ]:
# Initialize state
initial_state: TaskState = {
    "task": "Write a short blog post about AI agents",
    "steps": [],
    "current_step": "",
    "result": ""
}

# Run the graph
final_state = await task_graph.arun(initial_state)

# Print results
print("\nFinal State:")
print("Steps:", final_state["steps"])
print("Result:", final_state["result"])

## Stream Results

Now let's see how to stream results from the graph.

In [ ]:
async for event in task_graph.astream(initial_state):
    print("Event:", event)